In [2]:
# Banking Project - Data Cleaning and Preparation
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
import sys
import os

warnings.filterwarnings("ignore")

# Add shared modules to path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from shared.config.database import get_engine
from shared.utils.helpers import validate_data, VisualizationHelper

# Setup plotting style
VisualizationHelper.setup_plotting()

print("🏦 Banking Customer Churn Analysis - Data Cleaning")
print("=" * 60)

# Load data from CSV
customers = pd.read_csv("../../data/raw/banking_customers.csv")
transactions = pd.read_csv("../../data/raw/banking_transactions.csv")

# Alternatively, load from database
# engine = get_engine()
# customers = pd.read_sql('SELECT * FROM banking_customers', engine)
# transactions = pd.read_sql('SELECT * FROM banking_transactions', engine)

print("📁 Data Loaded Successfully!")
print(f"Customers shape: {customers.shape}")
print(f"Transactions shape: {transactions.shape}")

# Validate customers data
validate_data(customers, "Banking Customers")

print("\n" + "=" * 50)
print("CUSTOMERS DATA OVERVIEW")
print("=" * 50)
print(customers.head())
print(f"\nMissing values:\n{customers.isnull().sum()}")
print(f"\nData types:\n{customers.dtypes}")

# Validate transactions data
validate_data(transactions, "Banking Transactions")

print("\n" + "=" * 50)
print("TRANSACTIONS DATA OVERVIEW")
print("=" * 50)
print(transactions.head())
print(f"\nMissing values:\n{transactions.isnull().sum()}")
print(f"\nData types:\n{transactions.dtypes}")


def clean_customers_data(df):
    """Clean and prepare customers data"""
    df_clean = df.copy()

    # Handle data types
    df_clean["customer_id"] = df_clean["customer_id"].astype(str)

    # Check for duplicates
    print(f"Duplicate customers: {df_clean.duplicated().sum()}")

    # Validate ranges
    print("\n📊 Data Ranges Validation:")
    print(
        f"Credit Score: {df_clean['credit_score'].min()} - {df_clean['credit_score'].max()}"
    )
    print(f"Age: {df_clean['age'].min()} - {df_clean['age'].max()}")
    print(
        f"Balance: ${df_clean['balance'].min():,.2f} - ${df_clean['balance'].max():,.2f}"
    )

    # Fix any data issues
    df_clean = df_clean[df_clean["age"] >= 18]  # Remove unrealistic ages
    df_clean = df_clean[df_clean["balance"] >= 0]  # Remove negative balances

    return df_clean


def clean_transactions_data(df):
    """Clean and prepare transactions data"""
    df_clean = df.copy()

    # Handle data types
    df_clean["transaction_id"] = df_clean["transaction_id"].astype(str)
    df_clean["customer_id"] = df_clean["customer_id"].astype(str)
    df_clean["date"] = pd.to_datetime(df_clean["date"])

    # Check for duplicates
    print(f"Duplicate transactions: {df_clean.duplicated().sum()}")

    # Remove transactions with invalid amounts
    initial_count = len(df_clean)
    df_clean = df_clean[df_clean["amount"] != 0]
    print(f"Removed {initial_count - len(df_clean)} transactions with zero amount")

    return df_clean

# Clean the datasets
print("🧹 Cleaning Customers Data...")
customers_clean = clean_customers_data(customers)

print("\n🧹 Cleaning Transactions Data...")
transactions_clean = clean_transactions_data(transactions)

print(f"\n✅ Cleaning Complete!")
print(f"Original customers: {len(customers)} → Cleaned: {len(customers_clean)}")
print(
    f"Original transactions: {len(transactions)} → Cleaned: {len(transactions_clean)}"
)

# Create additional features for analysis
print("🔧 Creating Additional Features...")

# Customer transaction aggregations
customer_transactions = (
    transactions_clean.groupby("customer_id")
    .agg({"amount": ["count", "mean", "std", "min", "max"], "date": ["min", "max"]})
    .round(2)
)

customer_transactions.columns = [
    "txn_count",
    "avg_amount",
    "std_amount",
    "min_amount",
    "max_amount",
    "first_txn",
    "last_txn",
]
customer_transactions = customer_transactions.reset_index()

# Calculate days since last transaction
customer_transactions["days_since_last_txn"] = (
    pd.to_datetime("2024-01-01") - customer_transactions["last_txn"]
).dt.days

# Merge with customers data
customers_enriched = customers_clean.merge(
    customer_transactions, on="customer_id", how="left"
)

# Fill NaN values for customers with no transactions
customers_enriched["txn_count"] = customers_enriched["txn_count"].fillna(0)
customers_enriched["days_since_last_txn"] = customers_enriched[
    "days_since_last_txn"
].fillna(365)

print("✅ Feature Engineering Complete!")
print(f"Enriched customers shape: {customers_enriched.shape}")

# Save cleaned data
os.makedirs("../../data/processed", exist_ok=True)
customers_enriched.to_csv("../../data/processed/banking_customers_clean.csv", index=False)
transactions_clean.to_csv(
    "../../data/processed/banking_transactions_clean.csv", index=False
)

print("💾 Cleaned data saved to:")
print("   ../../data/processed/banking_customers_clean.csv")
print("   ../../data/processed/banking_transactions_clean.csv")

# Summary
print("\n🎯 DATA CLEANING SUMMARY")
print("=" * 40)
print(f"✅ Customers data: {len(customers_clean)} records")
print(f"✅ Transactions data: {len(transactions_clean)} records")
print(f"✅ Churn rate: {customers_enriched['churn'].mean():.2%}")
print(f"✅ Features created: {len(customers_enriched.columns)}")
print("\n📊 Key statistics:")
print(
    customers_enriched[
        ["credit_score", "age", "balance", "estimated_salary", "txn_count"]
    ].describe()
)

🏦 Banking Customer Churn Analysis - Data Cleaning
📁 Data Loaded Successfully!
Customers shape: (10000, 12)
Transactions shape: (50000, 8)

CUSTOMERS DATA OVERVIEW
  customer_id  credit_score         country  gender  age  tenure   balance  \
0   BANK10000           699   United States    Male   59       2   2955.00   
1   BANK10001           636   United States  Female   73       0  10438.29   
2   BANK10002           714         Germany  Female   34       4    469.00   
3   BANK10003           802  United Kingdom    Male   48       0  12339.78   
4   BANK10004           626  United Kingdom    Male   70       7  12984.43   

   products_number  credit_card  active_member  estimated_salary  churn  
0                4            0              0          43771.94      0  
1                1            1              0          41761.55      0  
2                3            1              1          42696.30      0  
3                3            1              1          17106.69      0 